<h1>Introduction</h1>

<p>Ce projet est un système de recommandation de livres intelligent basé sur l'apprentissage automatique (machine learning) qui utilise l'algorithme K-Means pour le clustering et une API REST Flask pour servir les recommandations. Le système est conçu pour analyser les préférences des utilisateurs et recommander des livres pertinents en fonction de leurs habitudes de lecture, de leurs évaluations passées et des caractéristiques des livres.</p>

<h1>Importations</h1>

In [56]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from flask import Flask, request, jsonify
import mysql.connector
from flask_cors import CORS
from mysql.connector import Error
from contextlib import contextmanager
import traceback
import warnings# Manipulation de données en tables (DataFrames)
import pandas as pd

# Calculs numériques, matrices, tableaux
import numpy as np

# Prétraitement : standardisation des données et encodage des labels catégoriels
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Algorithme de clustering non supervisé
from sklearn.cluster import KMeans

# Mesures d’évaluation de clustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Réduction de dimensionnalité
from sklearn.decomposition import PCA

# Visualisation graphique (plots, graphiques)
import matplotlib.pyplot as plt
import seaborn as sns

# Création d'une API web légère
from flask import Flask, request, jsonify

# Connexion à une base de données MySQL
import mysql.connector

# Autoriser les requêtes Cross-Origin (CORS) sur l’API Flask
from flask_cors import CORS

# Gestion des erreurs spécifiques à MySQL
from mysql.connector import Error

# Contexte manager pour gérer proprement les ressources (ex: connexions)
from contextlib import contextmanager

# Gestion et affichage des traces d’erreurs
import traceback

# Ignorer certains avertissements (warnings)
import warnings

# Manipulation des dates et heures
from datetime import datetime

# Lecture et écriture de données au format JSON
import json

# Désactiver les warnings pour un affichage plus propre
warnings.filterwarnings('ignore')

from datetime import datetime
import json
warnings.filterwarnings('ignore')

### Silhouette Score

Pour un point \( i \) :

- \( a(i) \) = moyenne des distances entre \( i \) et les autres points du même cluster.
- \( b(i) \) = plus petite moyenne des distances entre \( i \) et les autres clusters.

Le score de silhouette pour le point \( i \) est :

$$
s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$


### 2. Calinski-Harabasz Score (Indice de Calinski-Harabasz)

Considérons un jeu de données avec \( n \) points et \( k \) clusters.

- \( S_B \) = dispersion entre les clusters (between-cluster scatter matrix).
- \( S_W \) = dispersion à l’intérieur des clusters (within-cluster scatter matrix).

Le score est défini par :

$$
CH = \frac{\operatorname{trace}(S_B)}{\operatorname{trace}(S_W)} \times \frac{n - k}{k - 1}
$$

Plus ce score est élevé, meilleur est le clustering (clusters bien séparés et compacts).

---

### 3. Davies-Bouldin Score

Pour chaque cluster \( i \), on calcule :

- \( S_i \) = mesure de dispersion (ex. moyenne des distances des points du cluster \( i \) à son centre).
- \( M_{ij} \) = distance entre les centres des clusters \( i \) et \( j \).

Pour chaque cluster \( i \), on définit :

$$
R_{ij} = \frac{S_i + S_j}{M_{ij}}
$$

Le score de Davies-Bouldin pour le cluster \( i \) est :

$$
R_i = \max_{j \neq i} R_{ij}
$$

Le score global est la moyenne sur tous les clusters :

$$
DB = \frac{1}{k} \sum_{i=1}^k R_i
$$

Un score DB plus faible indique un meilleur clustering (clusters bien séparés et peu dispersés).


<h1>Configuration Initiale</h1>

In [57]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'reco_db'
}

<h1>Variables globales (initialisation)</h1>

In [58]:
books_df = None
ratings_df = None
users_df = None
cart_df = None
user_features = {}
book_features_matrix = None
book_id_to_index = {}
index_to_book_id = {}
book_clusters = {}
kmeans_model = None
scaler = StandardScaler()
category_encoder = LabelEncoder()
author_encoder = LabelEncoder()
data_quality_report = {}
evaluation_history = []
feature_names = []

<h1>Fonctions d'Analyse de Données</h1>

In [59]:
def analyze_data_quality():
    """Analyse la qualité des données"""
    print(" Analyse de la qualité des données...")
    
    quality_report = {
        'books': {},
        'ratings': {},
        'users': {},
        'issues': []
    }
    
    # Analyse des livres
    if books_df is not None and not books_df.empty:
        quality_report['books']['total'] = len(books_df)
        quality_report['books']['missing_values'] = books_df.isnull().sum().to_dict()
        quality_report['books']['duplicates'] = books_df.duplicated(subset=['title', 'author']).sum()
        
        # Détecter les problèmes
        if 'price' in books_df.columns:
            if books_df['price'].min() <= 0:
                quality_report['issues'].append("Prix non valides (≤ 0) détectés")
        if 'stock' in books_df.columns:
            if books_df['stock'].min() < 0:
                quality_report['issues'].append("Stocks négatifs détectés")
    
    # Analyse des évaluations
    if ratings_df is not None and not ratings_df.empty:
        quality_report['ratings']['total'] = len(ratings_df)
        quality_report['ratings']['missing_values'] = ratings_df.isnull().sum().to_dict()
        quality_report['ratings']['rating_range'] = {
            'min': ratings_df['rating'].min(),
            'max': ratings_df['rating'].max(),
            'mean': ratings_df['rating'].mean()
        }
        quality_report['ratings']['unique_users'] = ratings_df['user_id'].nunique()
        quality_report['ratings']['unique_books'] = ratings_df['book_id'].nunique()
        
        # Vérifier les évaluations non valides
        invalid_ratings = ratings_df[(ratings_df['rating'] < 1) | (ratings_df['rating'] > 5)]
        if len(invalid_ratings) > 0:
            quality_report['issues'].append(f"{len(invalid_ratings)} évaluations hors plage [1,5]")
    
    # Analyse des utilisateurs
    if users_df is not None and not users_df.empty:
        quality_report['users']['total'] = len(users_df)
        quality_report['users']['missing_values'] = users_df.isnull().sum().to_dict()
    
    print(f" Analyse qualité terminée: {len(quality_report['issues'])} problèmes détectés")
    return quality_report


<h1>Nettoyage des données</h1>

In [60]:
def clean_books_data(df):
    """Nettoyage avancé des données livres"""
    print(" Nettoyage des données livres...")
    
    # Copie pour éviter les warnings
    cleaned_df = df.copy()
    
    # 1. Gestion des valeurs manquantes
    if 'price' in cleaned_df.columns:
        cleaned_df['price'] = pd.to_numeric(cleaned_df['price'], errors='coerce')
        median_price = cleaned_df['price'].median()
        cleaned_df['price'] = cleaned_df['price'].fillna(median_price)
        
        # Remplacer les prix non valides
        cleaned_df.loc[cleaned_df['price'] <= 0, 'price'] = median_price
    
    if 'stock' in cleaned_df.columns:
        cleaned_df['stock'] = pd.to_numeric(cleaned_df['stock'], errors='coerce')
        median_stock = cleaned_df['stock'].median()
        cleaned_df['stock'] = cleaned_df['stock'].fillna(median_stock)
        cleaned_df.loc[cleaned_df['stock'] < 0, 'stock'] = 0
    
    # 2. Normalisation du texte
    if 'title' in cleaned_df.columns:
        cleaned_df['title'] = cleaned_df['title'].str.strip().str.title()
    
    if 'author' in cleaned_df.columns:
        cleaned_df['author'] = cleaned_df['author'].str.strip().str.title()
        
        # Encodage des auteurs
        try:
            cleaned_df['author_encoded'] = author_encoder.fit_transform(cleaned_df['author'])
        except:
            cleaned_df['author_encoded'] = 0
    
    # 3. Gestion des catégories
    if 'category_id' in cleaned_df.columns:
        # Remplacer les catégories manquantes
        mode_category = cleaned_df['category_id'].mode()[0] if not cleaned_df['category_id'].mode().empty else 1
        cleaned_df['category_id'] = cleaned_df['category_id'].fillna(mode_category)
        
        # Encodage
        cleaned_df['category_encoded'] = category_encoder.fit_transform(
            cleaned_df['category_id'].astype(str)
        )
    
    # 4. Extraction de features textuelles
    if 'title' in cleaned_df.columns:
        cleaned_df['title_length'] = cleaned_df['title'].str.len()
        cleaned_df['word_count'] = cleaned_df['title'].str.split().str.len()
    
    # 5. Features temporelles si disponible
    if 'year' in cleaned_df.columns:
        cleaned_df['year'] = pd.to_numeric(cleaned_df['year'], errors='coerce')
        current_year = datetime.now().year
        cleaned_df['age'] = current_year - cleaned_df['year'].fillna(current_year)
        cleaned_df['age'] = cleaned_df['age'].clip(lower=0, upper=100)
    
    print(f" Données nettoyées: {len(cleaned_df)} livres")
    return cleaned_df

<h1>Extraction de features</h1>

In [61]:
def extract_book_features():
    """Extraction avancée de features pour les livres"""
    print(" Extraction de features avancées...")
    
    global feature_names, book_features_matrix, book_id_to_index, index_to_book_id
    
    features_list = []
    book_ids = []
    
    for _, book in books_df.iterrows():
        book_id = book['id']
        book_features = {}
        
        # 1. Features basiques
        book_features['price'] = float(book['price'])
        book_features['stock'] = float(book['stock'])
        book_features['category_encoded'] = float(book.get('category_encoded', 0))
        book_features['author_encoded'] = float(book.get('author_encoded', 0))
        
        # 2. Features de popularité
        if ratings_df is not None and not ratings_df.empty:
            book_ratings = ratings_df[ratings_df['book_id'] == book_id]
            rating_count = len(book_ratings)
            avg_rating = book_ratings['rating'].mean() if rating_count > 0 else 3.0
            
            book_features['avg_rating'] = float(avg_rating)
            book_features['rating_count'] = float(rating_count)
            book_features['popularity'] = float(avg_rating * np.log1p(rating_count + 1))
            
            # Variabilité des notes
            if rating_count > 1:
                rating_std = book_ratings['rating'].std()
                book_features['rating_std'] = float(rating_std)
            else:
                book_features['rating_std'] = 0.0
        else:
            book_features['avg_rating'] = 3.0
            book_features['rating_count'] = 0.0
            book_features['popularity'] = 0.0
            book_features['rating_std'] = 0.0
        
        # 3. Features textuelles
        book_features['title_length'] = float(book.get('title_length', 0))
        book_features['word_count'] = float(book.get('word_count', 0))
        
        # 4. Features temporelles
        book_features['age'] = float(book.get('age', 0))
        
        # 5. Feature de disponibilité
        book_features['available'] = float(book['stock'] > 0) if 'stock' in book else 1.0
        
        # Convertir en liste dans l'ordre fixe
        feature_names = [
            'price', 'stock', 'category_encoded', 'author_encoded',
            'avg_rating', 'rating_count', 'popularity', 'rating_std',
            'title_length', 'word_count', 'age', 'available'
        ]
        
        features_vector = [book_features.get(name, 0.0) for name in feature_names]
        
        features_list.append(features_vector)
        book_ids.append(book_id)
    
    features_matrix = np.array(features_list)
    
    # Normalisation
    if len(features_list) > 1:
        features_matrix = scaler.fit_transform(features_matrix)
    
    # Réduction de dimensionnalité
    if features_matrix.shape[1] > 8:
        pca = PCA(n_components=8)
        features_matrix = pca.fit_transform(features_matrix)
        print(f" PCA appliqué: {features_matrix.shape[1]} composantes principales")
    
    book_features_matrix = features_matrix
    book_id_to_index = {book_id: idx for idx, book_id in enumerate(book_ids)}
    index_to_book_id = {idx: book_id for idx, book_id in enumerate(book_ids)}
    
    print(f"{len(feature_names)} features extraites pour {len(book_ids)} livres")
    
    return features_matrix

<h1>Détection des outliers</h1>

In [62]:
def detect_outliers(features_matrix, threshold=3):
    """Détection des outliers"""
    print("Détection des outliers...")
    
    # Méthode simple: distance à la médiane
    median = np.median(features_matrix, axis=0)
    mad = np.median(np.abs(features_matrix - median), axis=0)
    
    # Éviter la division par zéro
    mad[mad == 0] = 1
    
    # Score Z modifié
    modified_z_scores = 0.6745 * (features_matrix - median) / mad
    
    # Identifier les outliers
    outlier_mask = np.any(np.abs(modified_z_scores) > threshold, axis=1)
    outlier_count = np.sum(outlier_mask)
    
    print(f" {outlier_count} outliers détectés ({outlier_count/len(features_matrix):.1%})")
    
    return outlier_mask, outlier_count

<h1> Évaluation du clustering </h1>

In [63]:
def evaluate_clustering(features_matrix, labels, model_name="K-Means"):
    """Évalue la qualité du clustering"""
    print(f" Évaluation du clustering {model_name}...")
    
    if len(np.unique(labels)) < 2:
        print(" Pas assez de clusters pour l'évaluation")
        return {}
    
    metrics = {}
    
    try:
        # 1. Silhouette Score (-1 à 1, plus haut = mieux)
        silhouette = silhouette_score(features_matrix, labels)
        metrics['silhouette_score'] = float(silhouette)
        
        # 2. Calinski-Harabasz Index (plus haut = mieux)
        calinski = calinski_harabasz_score(features_matrix, labels)
        metrics['calinski_harabasz'] = float(calinski)
        
        # 3. Davies-Bouldin Index (plus bas = mieux)
        davies = davies_bouldin_score(features_matrix, labels)
        metrics['davies_bouldin'] = float(davies)
        
        # 4. Distribution des clusters
        unique, counts = np.unique(labels, return_counts=True)
        metrics['cluster_distribution'] = dict(zip(map(int, unique), map(int, counts)))
        metrics['n_clusters'] = int(len(unique))
        
        # 5. Taille des clusters
        metrics['cluster_sizes'] = {
            'min': int(np.min(counts)),
            'max': int(np.max(counts)),
            'mean': float(np.mean(counts)),
            'std': float(np.std(counts))
        }
        
        # 6. Coherence intra-cluster
        intra_cluster_distances = []
        for cluster_id in unique:
            cluster_points = features_matrix[labels == cluster_id]
            if len(cluster_points) > 1:
                centroid = np.mean(cluster_points, axis=0)
                distances = np.linalg.norm(cluster_points - centroid, axis=1)
                intra_cluster_distances.extend(distances)
        
        if intra_cluster_distances:
            metrics['intra_cluster_distance'] = {
                'mean': float(np.mean(intra_cluster_distances)),
                'std': float(np.std(intra_cluster_distances))
            }
        
        # Interprétation
        interpretations = []
        
        if silhouette > 0.7:
            interpretations.append("Clusters bien séparés et denses")
        elif silhouette > 0.5:
            interpretations.append("Clusters raisonnablement séparés")
        elif silhouette > 0.25:
            interpretations.append("Clusters faibles, certains points mal assignés")
        else:
            interpretations.append("Clusters peu définis")
        
        if davies < 0.5:
            interpretations.append("Faible chevauchement entre clusters")
        elif davies < 1.0:
            interpretations.append("Chevauchement modéré entre clusters")
        else:
            interpretations.append("Fort chevauchement entre clusters")
        
        metrics['interpretation'] = interpretations
        
        print(f" Évaluation terminée:")
        print(f"   Silhouette: {silhouette:.3f}")
        print(f"   Calinski-Harabasz: {calinski:.1f}")
        print(f"   Davies-Bouldin: {davies:.3f}")
        print(f"   {len(unique)} clusters, taille moyenne: {np.mean(counts):.1f}")
        
        # Stocker dans l'historique
        evaluation_history.append({
            'timestamp': datetime.now().isoformat(),
            'type': 'clustering',
            'metrics': metrics
        })
        
    except Exception as e:
        print(f"Erreur évaluation clustering: {e}")
        metrics['error'] = str(e)
    
    return metrics


<h1>Évalue la qualité des recommandations</h1>

In [64]:
def evaluate_recommendations(recommendations, user_history=None, all_books=None):
    """Évalue la qualité des recommandations"""
    print("Évaluation des recommandations...")
    
    if not recommendations:
        return {
            'coverage': 0,
            'diversity': 0,
            'novelty': 0,
            'serendipity': 0
        }
    
    metrics = {}
    
    # 1. Couverture (nombre unique de livres recommandés / total livres)
    if all_books is not None:
        recommended_ids = [r['id'] for r in recommendations if isinstance(r, dict) and 'id' in r]
        unique_recommended = set(recommended_ids)
        metrics['coverage'] = len(unique_recommended) / len(all_books) if len(all_books) > 0 else 0
        metrics['unique_recommended'] = len(unique_recommended)
        metrics['total_books'] = len(all_books)
    
    # 2. Diversité (basée sur les catégories)
    categories = []
    authors = []
    for rec in recommendations:
        if isinstance(rec, dict):
            if 'category_id' in rec:
                categories.append(rec['category_id'])
            if 'author' in rec:
                authors.append(rec['author'])
    
    if categories:
        metrics['category_diversity'] = len(set(categories)) / len(categories) if categories else 0
    if authors:
        metrics['author_diversity'] = len(set(authors)) / len(authors) if authors else 0
    
    # 3. Nouveauté (recommandations non vues par l'utilisateur)
    if user_history is not None:
        user_book_ids = set(user_history.get('evaluated_books', []))
        new_recommendations = [r for r in recommendations 
                             if isinstance(r, dict) and r.get('id') not in user_book_ids]
        metrics['novelty'] = len(new_recommendations) / len(recommendations) if recommendations else 0
    
    # 4. Score moyen
    if recommendations and 'score' in recommendations[0]:
        scores = [r.get('score', 0) for r in recommendations]
        metrics['score_distribution'] = {
            'min': float(np.min(scores)),
            'max': float(np.max(scores)),
            'mean': float(np.mean(scores)),
            'std': float(np.std(scores))
        }
    
    return metrics

<h1>Génère un rapport d'évaluation</h1>

In [65]:
def generate_evaluation_report(clustering_metrics=None, recommendation_metrics=None):
    """Génère un rapport d'évaluation"""
    print("Génération du rapport d'évaluation...")
    
    report = {
        'timestamp': datetime.now().isoformat(),
        'clustering': clustering_metrics or {},
        'recommendations': recommendation_metrics or {},
        'summary': {},
        'data_quality': data_quality_report
    }
    
    # Résumé clustering
    if clustering_metrics:
        report['summary']['clustering'] = {
            'n_clusters': clustering_metrics.get('n_clusters', 0),
            'silhouette_score': clustering_metrics.get('silhouette_score', 0),
            'quality': clustering_metrics.get('interpretation', ['Non évalué'])[0] if clustering_metrics.get('interpretation') else 'Non évalué'
        }
    
    # Résumé recommandations
    if recommendation_metrics:
        report['summary']['recommendations'] = {
            'coverage': recommendation_metrics.get('coverage', 0),
            'diversity': recommendation_metrics.get('category_diversity', 0),
            'novelty': recommendation_metrics.get('novelty', 0)
        }
    
    # Stocker dans l'historique
    evaluation_history.append({
        'timestamp': datetime.now().isoformat(),
        'type': 'full_report',
        'report': report
    })
    
    return report

<h1>Visualisation 2D des clusters</h1>

In [66]:
def visualize_clusters(features_matrix, labels, save_path="clusters_visualization.png"):
    """Visualisation 2D des clusters"""
    try:
        if features_matrix.shape[1] > 2:
            # Réduction à 2D avec PCA
            pca = PCA(n_components=2)
            reduced_data = pca.fit_transform(features_matrix)
        else:
            reduced_data = features_matrix
        
        plt.figure(figsize=(10, 8))
        
        # Scatter plot
        unique_labels = np.unique(labels)
        colors = plt.cm.Set3(np.linspace(0, 1, len(unique_labels)))
        
        for i, label in enumerate(unique_labels):
            cluster_points = reduced_data[labels == label]
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1],
                      color=colors[i], label=f'Cluster {label}', alpha=0.7)
        
        plt.title('Visualisation des Clusters K-Means')
        plt.xlabel('Composante principale 1')
        plt.ylabel('Composante principale 2')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f" Visualisation sauvegardée: {save_path}")
        
        plt.close()
        
        return True
        
    except Exception as e:
        print(f" Erreur visualisation: {e}")
        return False


<h1>Fonctions de Base de Données</h1>

In [67]:
@contextmanager
def get_connection():
    """Gestionnaire de connexion à la base de données"""
    connection = None
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        yield connection
    except Error as e:
        print(f" Erreur connexion MySQL: {e}")
        raise
    finally:
        if connection and connection.is_connected():
            connection.close()


<h1>Charge les données depuis la base MySQL"</h1>

In [68]:
def load_data_from_db():
    """Charge les données depuis la base MySQL"""
    global books_df, ratings_df, users_df, cart_df, data_quality_report
    
    try:
        print("Chargement des données depuis MySQL...")
        
        with get_connection() as conn:
            cursor = conn.cursor(dictionary=True)
            
            # 1. Charger les livres
            cursor.execute("""
                SELECT b.*, c.name as category_name 
                FROM books b 
                LEFT JOIN categories c ON b.category_id = c.id
            """)
            books = cursor.fetchall()
            books_df = pd.DataFrame(books)
            
            # 2. Charger les évaluations
            cursor.execute("SELECT user_id, book_id, rating FROM ratings")
            ratings = cursor.fetchall()
            ratings_df = pd.DataFrame(ratings) if ratings else pd.DataFrame(columns=['user_id', 'book_id', 'rating'])
            
            # 3. Charger les utilisateurs
            cursor.execute("SELECT id, username FROM users")
            users = cursor.fetchall()
            users_df = pd.DataFrame(users) if users else pd.DataFrame(columns=['id', 'username'])
            
            # 4. Charger le panier
            cursor.execute("SELECT user_id, book_id FROM cart")
            cart = cursor.fetchall()
            cart_df = pd.DataFrame(cart) if cart else pd.DataFrame(columns=['user_id', 'book_id'])
            
            print(f"Données brutes chargées: {len(books_df)} livres, {len(users_df)} utilisateurs, {len(ratings_df)} évaluations")
            
            # Analyse de la qualité des données
            data_quality_report = analyze_data_quality()
            
            # Prétraitement des données
            if len(books_df) > 0:
                books_df = clean_books_data(books_df)
                extract_book_features()
                train_kmeans_model()
                prepare_user_profiles()
            else:
                print(" Pas de livres dans la base de données")
                
    except Exception as e:
        print(f" Erreur chargement données: {e}")
        traceback.print_exc()

<h1>Fonctions du Modèle K-Means</h1>

In [69]:
def train_kmeans_model(n_clusters=None):
    """Entraîne le modèle K-Means"""
    global kmeans_model, book_clusters, book_features_matrix
    
    if book_features_matrix is None or len(book_features_matrix) < 2:
        print(" Pas assez de données pour K-Means")
        return False
    
    print(" Entraînement du modèle K-Means...")
    
    # Déterminer le nombre optimal de clusters
    if n_clusters is None:
        n_books = len(book_features_matrix)
        n_clusters = min(max(5, n_books // 5), 15)
    
    print(f" Utilisation de {n_clusters} clusters pour {len(book_features_matrix)} livres")
    
    kmeans_model = KMeans(
        n_clusters=n_clusters,
        random_state=42,
        n_init=10,
        max_iter=300
    )
    
    # Assigner les clusters
    clusters = kmeans_model.fit_predict(book_features_matrix)
    
    # Stocker les clusters par livre
    book_clusters = {}
    for idx, cluster_id in enumerate(clusters):
        book_id = index_to_book_id[idx]
        book_clusters[book_id] = int(cluster_id)
    
    # Évaluer le clustering
    clustering_metrics = evaluate_clustering(book_features_matrix, clusters)
    
    # Visualiser les clusters
    visualize_clusters(book_features_matrix, clusters)
    
    # Afficher la distribution
    cluster_counts = np.bincount(clusters)
    print(f" K-Means entraîné avec {n_clusters} clusters")
    print(f" Métriques: Silhouette={clustering_metrics.get('silhouette_score', 0):.3f}")
    for i, count in enumerate(cluster_counts):
        print(f"   Cluster {i}: {count} livres")
    
    return True


<h1>Fonctions de Profil Utilisateur</h1>

In [70]:
def prepare_user_profiles():
    """Prépare les profils utilisateurs depuis la base de données"""
    global user_features
    
    print(" Préparation des profils utilisateurs...")
    
    try:
        with get_connection() as conn:
            cursor = conn.cursor(dictionary=True)
            
            # Récupérer tous les utilisateurs
            cursor.execute("SELECT id FROM users")
            users = cursor.fetchall()
            
            for user in users:
                user_id = user['id']
                
                # Récupérer les évaluations de l'utilisateur
                cursor.execute("""
                    SELECT r.book_id, r.rating, b.category_id, b.author, b.price
                    FROM ratings r
                    JOIN books b ON r.book_id = b.id
                    WHERE r.user_id = %s
                """, (user_id,))
                ratings = cursor.fetchall()
                
                if len(ratings) >= 1:
                    ratings_df_local = pd.DataFrame(ratings)
                    
                    # Calcul des préférences
                    avg_rating = ratings_df_local['rating'].mean()
                    
                    # Catégories préférées
                    category_prefs = {}
                    for _, row in ratings_df_local.iterrows():
                        cat = row['category_id']
                        rating_val = row['rating']
                        category_prefs[cat] = category_prefs.get(cat, 0) + rating_val
                    
                    # Auteurs préférés
                    author_prefs = {}
                    for _, row in ratings_df_local.iterrows():
                        author = row['author']
                        rating_val = row['rating']
                        author_prefs[author] = author_prefs.get(author, 0) + rating_val
                    
                    # Plage de prix préférée
                    if not ratings_df_local.empty:
                        avg_price = ratings_df_local['price'].mean()
                        price_range = (max(0, avg_price * 0.5), avg_price * 1.5)
                    else:
                        price_range = (0, 100)
                    
                    user_features[user_id] = {
                        'avg_rating': float(avg_rating),
                        'category_prefs': dict(sorted(category_prefs.items(), key=lambda x: x[1], reverse=True)[:5]),
                        'author_prefs': dict(sorted(author_prefs.items(), key=lambda x: x[1], reverse=True)[:10]),
                        'preferred_price_range': price_range,
                        'total_ratings': len(ratings),
                        'last_updated': datetime.now().isoformat()
                    }
            
            print(f"Profils créés pour {len(user_features)} utilisateurs")
            
    except Exception as e:
        print(f"Erreur préparation profils: {e}")
        traceback.print_exc()

<h1>Recalcule le profil utilisateur</h1>

In [71]:
def recalculate_user_profile(user_id):
    """Recalcule le profil utilisateur"""
    global user_features
    
    try:
        with get_connection() as conn:
            cursor = conn.cursor(dictionary=True)
            cursor.execute("""
                SELECT r.book_id, r.rating, b.category_id, b.author, b.price
                FROM ratings r
                JOIN books b ON r.book_id = b.id
                WHERE r.user_id = %s
            """, (user_id,))
            ratings = cursor.fetchall()
            
            if ratings:
                ratings_df_local = pd.DataFrame(ratings)
                avg_rating = ratings_df_local['rating'].mean()
                
                category_prefs = {}
                author_prefs = {}
                
                for _, row in ratings_df_local.iterrows():
                    cat = row['category_id']
                    rating_val = row['rating']
                    category_prefs[cat] = category_prefs.get(cat, 0) + rating_val
                    
                    author = row['author']
                    author_prefs[author] = author_prefs.get(author, 0) + rating_val
                
                good_ratings = ratings_df_local[ratings_df_local['rating'] >= 4]
                if not good_ratings.empty:
                    avg_price = good_ratings['price'].mean()
                    price_range = (max(0, avg_price * 0.5), avg_price * 1.5)
                else:
                    price_range = (0, 100)
                
                user_features[user_id] = {
                    'avg_rating': float(avg_rating),
                    'category_prefs': dict(sorted(category_prefs.items(), key=lambda x: x[1], reverse=True)[:5]),
                    'author_prefs': dict(sorted(author_prefs.items(), key=lambda x: x[1], reverse=True)[:10]),
                    'preferred_price_range': price_range,
                    'total_ratings': len(ratings),
                    'last_updated': datetime.now().isoformat()
                }
                
                print(f"Profil {user_id} recalculé: {len(ratings)} évaluations")
            else:
                # Profil par défaut
                user_features[user_id] = {
                    'avg_rating': 3.0,
                    'category_prefs': {},
                    'author_prefs': {},
                    'preferred_price_range': (0, 100),
                    'total_ratings': 0,
                    'last_updated': datetime.now().isoformat()
                }
                
    except Exception as e:
        print(f" Erreur recalcul profil {user_id}: {e}")

<h1>Récupère le profil utilisateur</h1>

In [72]:
def get_user_profile(user_id):
    """Récupère le profil utilisateur"""
    if user_id in user_features:
        return user_features[user_id]
    
    recalculate_user_profile(user_id)
    
    if user_id in user_features:
        return user_features[user_id]
    
    return {
        'user_id': user_id,
        'avg_rating': 3.0,
        'category_prefs': {},
        'author_prefs': {},
        'preferred_price_range': (0, 100),
        'total_ratings': 0,
        'last_updated': datetime.now().isoformat()
    }

<h1>Fonctions de Recommandation</h1>

In [73]:
def get_recommendations(book_id, user_id=None, n_recommendations=8):
    """Recommandations pour un livre avec clustering K-Means"""
    if not kmeans_model or book_id not in book_clusters:
        print(f" K-Means non entraîné ou livre {book_id} non trouvé")
        return []
    
    try:
        # Trouver le cluster du livre
        book_cluster = book_clusters[book_id]
        
        # Trouver tous les livres du même cluster
        cluster_book_ids = []
        for bid, cid in book_clusters.items():
            if cid == book_cluster and bid != book_id:
                cluster_book_ids.append(bid)
        
        print(f" Livre {book_id} dans le cluster {book_cluster} - {len(cluster_book_ids)} livres similaires")
        
        # Si pas assez de livres dans le cluster, chercher dans les clusters similaires
        if len(cluster_book_ids) < n_recommendations:
            print(f"Pas assez de livres dans le cluster {book_cluster}, recherche étendue...")
            
            # Calculer la distance aux autres clusters
            book_idx = book_id_to_index[book_id]
            book_vector = book_features_matrix[book_idx].reshape(1, -1)
            
            # Trouver les distances aux centroïdes
            distances = kmeans_model.transform(book_vector)[0]
            
            # Trier les clusters par distance
            sorted_clusters = np.argsort(distances)
            
            # Prendre des livres des clusters les plus proches
            additional_books = []
            for cluster in sorted_clusters:
                if cluster == book_cluster or len(additional_books) >= n_recommendations:
                    continue
                
                # Trouver les livres de ce cluster
                cluster_books = [bid for bid, cid in book_clusters.items() if cid == cluster and bid != book_id]
                
                # Prendre quelques livres de ce cluster
                take_count = min(2, len(cluster_books))
                if take_count > 0:
                    import random
                    selected = random.sample(cluster_books, take_count)
                    additional_books.extend(selected)
            
            cluster_book_ids.extend(additional_books)
        
        # Limiter le nombre de livres
        cluster_book_ids = cluster_book_ids[:n_recommendations * 3]
        
        recommendations = []
        for rec_book_id in cluster_book_ids:
            book_data = books_df[books_df['id'] == rec_book_id].iloc[0]
            
            # Calculer la similarité
            same_cluster = book_clusters[rec_book_id] == book_cluster
            similarity = 0.9 if same_cluster else 0.6
            
            personalization_score = 0
            reason = f"Cluster similaire ({book_cluster})"
            
            if user_id and user_id in user_features:
                profile = user_features[user_id]
                
                if book_data['category_id'] in profile['category_prefs']:
                    personalization_score += profile['category_prefs'][book_data['category_id']] * 0.3
                    reason = "Catégorie préférée dans votre cluster"
                
                if book_data['author'] in profile['author_prefs']:
                    personalization_score += profile['author_prefs'][book_data['author']] * 0.5
                    reason = "Auteur que vous appréciez"
                
                min_price, max_price = profile['preferred_price_range']
                if min_price <= book_data['price'] <= max_price:
                    personalization_score += 0.2
            
            final_score = similarity * 0.7 + personalization_score * 0.3
            
            recommendations.append({
                'id': int(rec_book_id),
                'title': str(book_data['title']),
                'author': str(book_data['author']),
                'category_id': int(book_data['category_id']),
                'price': float(book_data['price']),
                'image_url': str(book_data.get('image_url', f'https://via.placeholder.com/220x300?text={book_data["title"]}')),
                'similarity': similarity,
                'personalization_score': personalization_score,
                'final_score': final_score,
                'cluster_id': int(book_clusters[rec_book_id]),
                'reason': reason
            })
        
        recommendations.sort(key=lambda x: x['final_score'], reverse=True)
        return recommendations[:n_recommendations]
        
    except Exception as e:
        print(f" Erreur recommandations K-Means: {e}")
        traceback.print_exc()
        return []


<h1>ecommandations intelligentes basées sur les préférences utilisateur</h1>

In [74]:
def get_intelligent_recommendations(user_id, n_recommendations=12):
    """Recommandations intelligentes basées sur les préférences utilisateur"""
    try:
        print(f" Début recommandations intelligentes pour user {user_id}")
        
        # Recharger les données à jour
        load_data_from_db()
        
        if books_df is None or books_df.empty:
            print(" Pas de livres dans la base")
            return get_enhanced_fallback_recommendations()
        
        print(f"{len(books_df)} livres disponibles dans {len(set(book_clusters.values()))} clusters")
        
        # Recalculer le profil
        recalculate_user_profile(user_id)
        
        # Récupérer le profil
        profile = get_user_profile(user_id)
        
        total_ratings = profile.get('total_ratings', 0)
        print(f"Profil user {user_id}: {total_ratings} évaluations")
        
        # Analyser les clusters préférés de l'utilisateur
        user_cluster_prefs = {}
        if total_ratings > 0 and ratings_df is not None:
            user_ratings = ratings_df[ratings_df['user_id'] == user_id]
            for _, rating_row in user_ratings.iterrows():
                book_id = rating_row['book_id']
                if book_id in book_clusters:
                    cluster_id = book_clusters[book_id]
                    rating = rating_row['rating']
                    user_cluster_prefs[cluster_id] = user_cluster_prefs.get(cluster_id, 0) + rating
        
        # Trouver les clusters préférés
        preferred_clusters = []
        if user_cluster_prefs:
            sorted_clusters = sorted(user_cluster_prefs.items(), key=lambda x: x[1], reverse=True)
            preferred_clusters = [cluster_id for cluster_id, _ in sorted_clusters[:3]]
            print(f" Clusters préférés: {preferred_clusters}")
        
        # TOUJOURS inclure des livres
        print(" Stratégie: Inclure TOUS les livres (même évalués) avec scores ajustés")
        
        recommended_books = []
        books_processed = 0
        
        # Livres évalués par l'utilisateur
        user_evaluated_books = set()
        if ratings_df is not None and not ratings_df.empty:
            user_ratings_data = ratings_df[ratings_df['user_id'] == user_id]
            user_evaluated_books = set(user_ratings_data['book_id'].tolist())
        
        print(f" Livres évalués par user {user_id}: {len(user_evaluated_books)}")
        
        # Parcourir TOUS les livres
        for _, book in books_df.iterrows():
            books_processed += 1
            score = 0
            reasons = []
            
            # Vérifier si l'utilisateur a déjà évalué ce livre
            is_evaluated = book['id'] in user_evaluated_books
            
            # Récupérer la note de l'utilisateur pour ce livre
            user_rating = None
            if is_evaluated and ratings_df is not None:
                user_ratings = ratings_df[
                    (ratings_df['user_id'] == user_id) & 
                    (ratings_df['book_id'] == book['id'])
                ]
                if not user_ratings.empty:
                    user_rating = user_ratings.iloc[0]['rating']
            
            # Score de cluster (30%)
            if book['id'] in book_clusters:
                cluster_id = book_clusters[book['id']]
                if cluster_id in preferred_clusters:
                    cluster_score = user_cluster_prefs.get(cluster_id, 1) * 0.3
                    score += cluster_score
                    reasons.append(f"Cluster préféré ({cluster_id})")
                else:
                    score += 0.1 * 0.3
            
            # Score de catégorie (20%)
            if 'category_prefs' in profile and book['category_id'] in profile['category_prefs']:
                cat_score = profile['category_prefs'][book['category_id']]
                if is_evaluated and user_rating:
                    if user_rating >= 4:
                        cat_score = cat_score * 1.2
                        reasons.append(f"Catégorie préférée (vous avez noté {user_rating}/5)")
                    elif user_rating <= 2:
                        cat_score = cat_score * 0.3
                    else:
                        cat_score = cat_score * 0.7
                else:
                    reasons.append("Catégorie préférée")
                
                score += cat_score * 0.2
            
            # Score d'auteur (20%)
            if 'author_prefs' in profile and book['author'] in profile['author_prefs']:
                author_score = profile['author_prefs'][book['author']]
                if is_evaluated and user_rating:
                    if user_rating >= 4:
                        author_score = author_score * 1.3
                        reasons.append(f"Auteur apprécié (vous avez noté {user_rating}/5)")
                    elif user_rating <= 2:
                        author_score = author_score * 0.2
                    else:
                        author_score = author_score * 0.6
                else:
                    reasons.append("Auteur apprécié")
                
                score += author_score * 0.2
            
            # Score de prix (10%)
            if 'preferred_price_range' in profile:
                min_price, max_price = profile['preferred_price_range']
                if min_price <= book['price'] <= max_price:
                    score += 0.1
                    reasons.append("Budget adapté")
            
            # Score de popularité globale (10%)
            if ratings_df is not None and not ratings_df.empty:
                book_ratings = ratings_df[ratings_df['book_id'] == book['id']]
                if not book_ratings.empty:
                    avg_book_rating = book_ratings['rating'].mean()
                    rating_count = len(book_ratings)
                    popularity_score = avg_book_rating * np.log1p(rating_count + 1)
                    score += popularity_score * 0.1
                    if avg_book_rating >= 4.0:
                        reasons.append(f"Note: {avg_book_rating:.1f}/5")
            
            # Score de nouveauté (10%)
            novelty_score = 0.1 * (books_processed % 10) / 10
            score += novelty_score
            
            # Score minimal pour TOUS les livres
            base_score = 0.05
            score += base_score
            
            # Récupérer les notes moyennes
            avg_book_rating_val = 3.0
            rating_count_val = 0
            if ratings_df is not None and not ratings_df.empty:
                book_ratings = ratings_df[ratings_df['book_id'] == book['id']]
                if not book_ratings.empty:
                    avg_book_rating_val = book_ratings['rating'].mean()
                    rating_count_val = len(book_ratings)
            
            recommended_books.append({
                'id': int(book['id']),
                'title': str(book['title']),
                'author': str(book['author']),
                'score': float(score),
                'reasons': reasons,
                'category_id': int(book['category_id']),
                'price': float(book['price']),
                'image_url': book.get('image_url', f'https://via.placeholder.com/220x300?text={book["title"]}'),
                'is_evaluated': is_evaluated,
                'user_rating': float(user_rating) if user_rating else None,
                'avg_rating': float(avg_book_rating_val),
                'rating_count': int(rating_count_val)
            })
        
        print(f"{len(recommended_books)} livres évalués pour recommandation")
        
        # Trier par score
        recommended_books.sort(key=lambda x: x['score'], reverse=True)
        
        # Appliquer diversité intelligente
        final_recommendations = []
        categories_used = set()
        authors_used = set()
        clusters_used = set()
        
        # Mélanger livres évalués et non évalués
        evaluated_included = 0
        non_evaluated_included = 0
        
        for book in recommended_books:
            if len(final_recommendations) >= n_recommendations:
                break
            
            # Critères de diversité
            category_ok = (len(categories_used) < 4 or book['category_id'] not in categories_used)
            author_ok = (len(authors_used) < 6 or book['author'] not in authors_used)
            
            # Vérifier le cluster
            cluster_id = book_clusters.get(book['id'])
            cluster_ok = True
            if cluster_id is not None:
                cluster_books = [b for b in final_recommendations if book_clusters.get(b['id']) == cluster_id]
                if len(cluster_books) >= 3:
                    cluster_ok = False
            
            if (category_ok or author_ok) and cluster_ok:
                # Générer une raison personnalisée
                if book['is_evaluated']:
                    if book['user_rating']:
                        if book['user_rating'] >= 4:
                            reason = f" Vous avez adoré ({book['user_rating']}/5) - {format_reasons(book['reasons'])}"
                        elif book['user_rating'] <= 2:
                            reason = f"↩ Vous pourriez réévaluer ({book['user_rating']}/5)"
                        else:
                            reason = f" Déjà lu ({book['user_rating']}/5) - {format_reasons(book['reasons'])}"
                    else:
                        reason = "Déjà consulté - " + format_reasons(book['reasons'])
                    evaluated_included += 1
                else:
                    if book['avg_rating'] >= 4.5 and book['rating_count'] > 5:
                        reason = f" Excellent livre ({book['avg_rating']:.1f}/5) - " + format_reasons(book['reasons'])
                    elif book['avg_rating'] >= 4.0:
                        reason = f" Très bien noté ({book['avg_rating']:.1f}/5) - " + format_reasons(book['reasons'])
                    else:
                        reason = " Recommandation personnalisée - " + format_reasons(book['reasons'])
                    non_evaluated_included += 1
                
                final_recommendations.append({
                    **book,
                    'reason': reason or "Recommandé pour vous"
                })
                
                categories_used.add(book['category_id'])
                authors_used.add(book['author'])
                if cluster_id is not None:
                    clusters_used.add(cluster_id)
        
        # GARANTIE: Si moins de recommandations que demandé, prendre les meilleurs restants
        if len(final_recommendations) < n_recommendations:
            print(f" Seulement {len(final_recommendations)} recommandations après diversité, ajout des meilleurs restants")
            for book in recommended_books:
                if book['id'] not in [r['id'] for r in final_recommendations]:
                    reason = book.get('reason', 'Suggestion basée sur nos collections')
                    if not reason and book['is_evaluated'] and book['user_rating']:
                        reason = f"Vous avez noté {book['user_rating']}/5"
                    
                    final_recommendations.append({
                        **book,
                        'reason': reason or "Suggestion personnalisée"
                    })
                
                if len(final_recommendations) >= n_recommendations:
                    break
        
        print(f" {len(final_recommendations)} recommandations FINALES pour user {user_id} "
              f"({evaluated_included} déjà évalués, {non_evaluated_included} nouveaux)")
        
        return final_recommendations
        
    except Exception as e:
        print(f"Erreur recommandations intelligentes: {e}")
        traceback.print_exc()
        return get_enhanced_fallback_recommendations()


<h1>Fallback amélioré qui retourne TOUJOURS des livres</h1>

In [75]:
def get_enhanced_fallback_recommendations():
    """Fallback amélioré qui retourne TOUJOURS des livres"""
    try:
        print("Utilisation du fallback amélioré")
        
        if books_df is None or books_df.empty:
            print("Base de données vide dans fallback")
            return []
        
        fallback_books = []
        
        for idx, book in books_df.iterrows():
            if len(fallback_books) >= 12:
                break
            
            # Calculer un score basique GARANTI
            score = 0.5 + (idx % 10) * 0.05
            
            # Score de popularité
            if ratings_df is not None and not ratings_df.empty:
                book_ratings = ratings_df[ratings_df['book_id'] == book['id']]
                if not book_ratings.empty:
                    avg_rating = book_ratings['rating'].mean()
                    rating_count = len(book_ratings)
                    score += avg_rating * 0.2
                    score += np.log1p(rating_count) * 0.1
            
            # Score minimal
            if score < 0.3:
                score = 0.3
            
            # Récupérer la note moyenne
            avg_rating = 3.0
            rating_count = 0
            if ratings_df is not None and not ratings_df.empty:
                book_ratings = ratings_df[ratings_df['book_id'] == book['id']]
                if not book_ratings.empty:
                    avg_rating = book_ratings['rating'].mean()
                    rating_count = len(book_ratings)
            
            fallback_books.append({
                'id': int(book['id']),
                'title': str(book['title']),
                'author': str(book['author']),
                'category_id': int(book['category_id']),
                'price': float(book['price']),
                'image_url': book.get('image_url', f'https://via.placeholder.com/220x300?text={book["title"]}'),
                'score': float(score),
                'avg_rating': float(avg_rating),
                'rating_count': int(rating_count),
                'reason': f"Livre populaire ({avg_rating:.1f}/5)" if rating_count > 0 else "Découverte recommandée"
            })
        
        # GARANTIE: Toujours retourner au moins 12 livres
        if len(fallback_books) < 12:
            print(f" Seulement {len(fallback_books)} livres, duplicata autorisé")
            while len(fallback_books) < 12 and len(fallback_books) > 0:
                for book in fallback_books.copy():
                    if len(fallback_books) >= 12:
                        break
                    new_book = book.copy()
                    new_book['reason'] = "Suggestion supplémentaire"
                    fallback_books.append(new_book)
        
        # Trier par score
        fallback_books.sort(key=lambda x: x['score'], reverse=True)
        
        print(f"{len(fallback_books)} livres dans fallback")
        return fallback_books[:12]
        
    except Exception as e:
        print(f"Erreur fallback: {e}")
        return [
            {
                'id': 999,
                'title': 'Livre recommandé',
                'author': 'Auteur',
                'category_id': 1,
                'price': 19.99,
                'image_url': 'https://via.placeholder.com/220x300?text=Livre',
                'reason': 'Recommandation système'
            }
        ]

<h1>Formate les raisons pour l'affichage</h1>

In [76]:
def format_reasons(reasons):
    """Formate les raisons pour l'affichage"""
    if not reasons:
        return "Recommandé pour vous"
    
    if len(reasons) > 2:
        return f"{reasons[0]}, {reasons[1]}"
    return ", ".join(reasons)

<h1>Exécution et Test</h1>

In [77]:
print("Démarrage du système de recommandation...")

# Charger les données
load_data_from_db()

# Exécuter une évaluation automatique au démarrage
if book_clusters and book_features_matrix is not None:
    print("\n" + "="*50)
    print("ÉVALUATION AUTOMATIQUE AU DÉMARRAGE")
    print("="*50)
    
    clusters = []
    valid_indices = []
    for idx, book_id in index_to_book_id.items():
        cluster_id = book_clusters.get(book_id, -1)
        if cluster_id != -1:
            clusters.append(cluster_id)
            valid_indices.append(idx)
    
    if clusters:
        clusters_array = np.array(clusters)
        features_subset = book_features_matrix[valid_indices]
        
        if len(np.unique(clusters_array)) > 1:
            metrics = evaluate_clustering(features_subset, clusters_array)
            print(f"\nRésumé clustering:")
            print(f"   Score Silhouette: {metrics.get('silhouette_score', 0):.3f}")
            print(f"   Davies-Bouldin: {metrics.get('davies_bouldin', 0):.3f}")
            print(f"   {metrics.get('n_clusters', 0)} clusters")
    
    # Afficher le rapport de qualité
    if data_quality_report:
        print(f"\nQualité des données:")
        print(f"   Livres: {data_quality_report.get('books', {}).get('total', 0)}")
        print(f"   Évaluations: {data_quality_report.get('ratings', {}).get('total', 0)}")
        if 'issues' in data_quality_report:
            print(f"   Problèmes détectés: {len(data_quality_report['issues'])}")
            for issue in data_quality_report['issues']:
                print(f"     - {issue}")

print("\nSystème prêt pour les tests!")


Démarrage du système de recommandation...
Chargement des données depuis MySQL...
Données brutes chargées: 503 livres, 6 utilisateurs, 39 évaluations
 Analyse de la qualité des données...
 Analyse qualité terminée: 0 problèmes détectés
 Nettoyage des données livres...
 Données nettoyées: 503 livres
 Extraction de features avancées...
 PCA appliqué: 8 composantes principales
12 features extraites pour 503 livres
 Entraînement du modèle K-Means...
 Utilisation de 15 clusters pour 503 livres
 Évaluation du clustering K-Means...
 Évaluation terminée:
   Silhouette: 0.242
   Calinski-Harabasz: 129.1
   Davies-Bouldin: 1.096
   15 clusters, taille moyenne: 33.5
 Visualisation sauvegardée: clusters_visualization.png
 K-Means entraîné avec 15 clusters
 Métriques: Silhouette=0.242
   Cluster 0: 72 livres
   Cluster 1: 4 livres
   Cluster 2: 48 livres
   Cluster 3: 66 livres
   Cluster 4: 2 livres
   Cluster 5: 57 livres
   Cluster 6: 69 livres
   Cluster 7: 53 livres
   Cluster 8: 3 livres
   Cl

<h1>Tests et Démonstrations</h1>

In [78]:
user_id_test = 1
profile = get_user_profile(user_id_test)
print(f"\n Profil utilisateur {user_id_test}:")
print(f"   Nombre d'évaluations: {profile.get('total_ratings', 0)}")
print(f"   Note moyenne: {profile.get('avg_rating', 0):.2f}")
print(f"   Catégories préférées: {list(profile.get('category_prefs', {}).keys())[:3]}")


 Profil utilisateur 1:
   Nombre d'évaluations: 3
   Note moyenne: 4.00
   Catégories préférées: [3, 4, 1]


<h2> Recommandations pour un livre (remplacez 1 par un ID livre existant)
</h2>

In [79]:
book_id_test = 1
recommendations = get_recommendations(book_id_test, user_id=user_id_test, n_recommendations=5)
print(f"\nRecommandations pour le livre {book_id_test}:")
for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec['title']} - Score: {rec['final_score']:.3f}")

 Livre 1 dans le cluster 8 - 2 livres similaires
Pas assez de livres dans le cluster 8, recherche étendue...

Recommandations pour le livre 1:
   1. 1984 - Score: 1.650
   2. Candide - Score: 1.410
   3. Fondation - Score: 0.840
   4. Les Misérables - Score: 0.750
   5. Orgeuil Et Préjugés - Score: 0.750


<h1>Recommandations intelligentes pour utilisateur

In [80]:
intelligent_recs = get_intelligent_recommendations(user_id_test, n_recommendations=8)
print(f"\nRecommandations intelligentes pour utilisateur {user_id_test}:")
for i, rec in enumerate(intelligent_recs, 1):
    print(f"   {i}. {rec['title']} - Score: {rec['score']:.3f}")
    print(f"      Raison: {rec['reason']}")

 Début recommandations intelligentes pour user 1
Chargement des données depuis MySQL...
Données brutes chargées: 503 livres, 6 utilisateurs, 39 évaluations
 Analyse de la qualité des données...
 Analyse qualité terminée: 0 problèmes détectés
 Nettoyage des données livres...
 Données nettoyées: 503 livres
 Extraction de features avancées...
 PCA appliqué: 8 composantes principales
12 features extraites pour 503 livres
 Entraînement du modèle K-Means...
 Utilisation de 15 clusters pour 503 livres
 Évaluation du clustering K-Means...
 Évaluation terminée:
   Silhouette: 0.242
   Calinski-Harabasz: 129.1
   Davies-Bouldin: 1.096
   15 clusters, taille moyenne: 33.5
 Visualisation sauvegardée: clusters_visualization.png
 K-Means entraîné avec 15 clusters
 Métriques: Silhouette=0.242
   Cluster 0: 72 livres
   Cluster 1: 4 livres
   Cluster 2: 48 livres
   Cluster 3: 66 livres
   Cluster 4: 2 livres
   Cluster 5: 57 livres
   Cluster 6: 69 livres
   Cluster 7: 53 livres
   Cluster 8: 3 livre

<h1> Évaluation complète pour un utilisateur

In [81]:
def run_full_evaluation(user_id):
    """Exécute une évaluation complète pour un utilisateur"""
    print(f"\n ÉVALUATION COMPLÈTE POUR UTILISATEUR {user_id}")
    print("="*60)
    
    # 1. Évaluer le clustering
    clustering_metrics = None
    if book_features_matrix is not None and book_clusters:
        clusters = []
        valid_indices = []
        for idx, book_id in index_to_book_id.items():
            cluster_id = book_clusters.get(book_id, -1)
            if cluster_id != -1:
                clusters.append(cluster_id)
                valid_indices.append(idx)
        
        if clusters:
            clusters_array = np.array(clusters)
            features_subset = book_features_matrix[valid_indices]
            clustering_metrics = evaluate_clustering(features_subset, clusters_array)
    
    # 2. Générer et évaluer les recommandations
    recommendations = get_intelligent_recommendations(user_id)
    
    # Récupérer l'historique
    user_history = {'evaluated_books': []}
    if ratings_df is not None:
        user_ratings = ratings_df[ratings_df['user_id'] == user_id]
        user_history['evaluated_books'] = user_ratings['book_id'].tolist()
    
    rec_metrics = evaluate_recommendations(
        recommendations,
        user_history=user_history,
        all_books=books_df['id'].tolist() if books_df is not None else []
    )
    
    # 3. Générer un rapport
    report = generate_evaluation_report(
        clustering_metrics=clustering_metrics,
        recommendation_metrics=rec_metrics
    )
    
    print(f"\nRÉSULTATS:")
    if clustering_metrics:
        print(f"   - Score Silhouette: {clustering_metrics.get('silhouette_score', 0):.3f}")
    print(f"   - Couverture recommandations: {rec_metrics.get('coverage', 0):.1%}")
    print(f"   - Diversité catégories: {rec_metrics.get('category_diversity', 0):.1%}")
    print(f"   - Nouveauté: {rec_metrics.get('novelty', 0):.1%}")
    
    return report

# Exécuter l'évaluation
evaluation_report = run_full_evaluation(user_id_test)

# %%
print("\n NOTEBOOK TERMINÉ AVEC SUCCÈS!")
print("Vous pouvez maintenant exécuter différentes cellules pour tester le système.")


 ÉVALUATION COMPLÈTE POUR UTILISATEUR 1
 Évaluation du clustering K-Means...
 Évaluation terminée:
   Silhouette: 0.242
   Calinski-Harabasz: 129.1
   Davies-Bouldin: 1.096
   15 clusters, taille moyenne: 33.5
 Début recommandations intelligentes pour user 1
Chargement des données depuis MySQL...
Données brutes chargées: 503 livres, 6 utilisateurs, 39 évaluations
 Analyse de la qualité des données...
 Analyse qualité terminée: 0 problèmes détectés
 Nettoyage des données livres...
 Données nettoyées: 503 livres
 Extraction de features avancées...
 PCA appliqué: 8 composantes principales
12 features extraites pour 503 livres
 Entraînement du modèle K-Means...
 Utilisation de 15 clusters pour 503 livres
 Évaluation du clustering K-Means...
 Évaluation terminée:
   Silhouette: 0.242
   Calinski-Harabasz: 129.1
   Davies-Bouldin: 1.096
   15 clusters, taille moyenne: 33.5
 Visualisation sauvegardée: clusters_visualization.png
 K-Means entraîné avec 15 clusters
 Métriques: Silhouette=0.242


In [ ]:



app = Flask(__name__)
CORS(app)

# %%
@app.route('/')
def home():
    return "API de Recommandation avec MySQL, K-Means et Évaluation 🚀"

# %%
@app.route('/health')
def health():
    return jsonify({
        'status': 'healthy',
        'books': len(books_df) if books_df is not None else 0,
        'users_with_profiles': len(user_features),
        'clusters': len(set(book_clusters.values())) if book_clusters else 0,
        'model': 'K-Means',
        'evaluation_ready': len(evaluation_history) > 0
    })

# %%
@app.route('/initialize', methods=['POST'])
def initialize():
    """Initialise le modèle"""
    try:
        load_data_from_db()
        return jsonify({
            'status': 'success',
            'message': f'Modèle initialisé avec {len(books_df)} livres et {len(user_features)} profils',
            'clusters': len(set(book_clusters.values())) if book_clusters else 0,
            'data_quality_issues': len(data_quality_report.get('issues', []))
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/recommend/<int:book_id>', methods=['GET'])
def recommend(book_id):
    """Recommandations pour un livre"""
    try:
        user_id = request.args.get('user_id', type=int)
        
        recommendations = get_recommendations(
            book_id, 
            user_id=user_id,
            n_recommendations=8
        )
        
        return jsonify({
            'status': 'success',
            'recommendations': recommendations,
            'count': len(recommendations),
            'model': 'K-Means'
        })
    except Exception as e:
        print(f" Erreur route /recommend: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/intelligent-recommendations/<int:user_id>', methods=['GET'])
def intelligent_recommendations(user_id):
    """Recommandations intelligentes basées sur les préférences"""
    try:
        print(f"Début recommandations intelligentes pour user {user_id}")
        
        # Forcer le chargement des données
        load_data_from_db()
        
        if books_df is None or books_df.empty:
            print("Pas de livres dans la base")
            return jsonify({
                'status': 'success',
                'recommendations': [],
                'count': 0,
                'message': 'Base de données vide'
            })
        
        print(f"{len(books_df)} livres disponibles")
        
        # Obtenir les recommandations
        recommendations = get_intelligent_recommendations(user_id)
        
        print(f" {len(recommendations)} recommandations générées")
        
        # Formater la réponse
        formatted_recommendations = []
        for rec in recommendations:
            formatted_rec = {
                'id': rec['id'],
                'title': rec['title'],
                'author': rec['author'],
                'reason': rec.get('reason', 'Recommandé pour vous'),
                'category_id': rec.get('category_id', 0),
                'price': rec.get('price', 0),
                'image_url': rec.get('image_url', f'https://via.placeholder.com/220x300?text={rec["title"]}'),
                'score': rec.get('score', 0.5)
            }
            formatted_recommendations.append(formatted_rec)
        
        return jsonify({
            'status': 'success',
            'recommendations': formatted_recommendations,
            'count': len(formatted_recommendations),
            'algorithm': 'intelligent',
            'model': 'K-Means',
            'timestamp': datetime.now().isoformat(),
            'books_in_db': len(books_df)
        })
        
    except Exception as e:
        print(f"Erreur route recommandations intelligentes: {e}")
        traceback.print_exc()
        
        # Fallback
        fallback_recs = get_enhanced_fallback_recommendations()
        
        return jsonify({
            'status': 'success',
            'recommendations': fallback_recs,
            'count': len(fallback_recs),
            'algorithm': 'fallback',
            'timestamp': datetime.now().isoformat()
        })

# %%
@app.route('/data-quality', methods=['GET'])
def get_data_quality():
    """Retourne le rapport de qualité des données"""
    try:
        return jsonify({
            'status': 'success',
            'quality_report': data_quality_report
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/evaluate/clustering', methods=['GET'])
def evaluate_clustering_route():
    """Évalue le clustering"""
    try:
        if book_features_matrix is None or not book_clusters:
            return jsonify({'status': 'error', 'message': 'Clustering non disponible'}), 400
        
        # Préparer les données
        clusters = []
        valid_indices = []
        
        for idx, book_id in index_to_book_id.items():
            cluster_id = book_clusters.get(book_id, -1)
            if cluster_id != -1:
                clusters.append(cluster_id)
                valid_indices.append(idx)
        
        if not clusters:
            return jsonify({'status': 'error', 'message': 'Aucun cluster valide'}), 400
        
        clusters_array = np.array(clusters)
        features_subset = book_features_matrix[valid_indices]
        
        # Évaluer
        metrics = evaluate_clustering(features_subset, clusters_array)
        
        return jsonify({
            'status': 'success',
            'metrics': metrics,
            'model': 'K-Means'
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/evaluate/recommendations/<int:user_id>', methods=['GET'])
def evaluate_user_recommendations(user_id):
    """Évalue les recommandations pour un utilisateur"""
    try:
        n_recommendations = request.args.get('n', default=10, type=int)
        
        # Générer les recommandations
        recommendations = get_intelligent_recommendations(user_id, n_recommendations)
        
        # Récupérer l'historique
        user_history = {'evaluated_books': []}
        if ratings_df is not None:
            user_ratings = ratings_df[ratings_df['user_id'] == user_id]
            user_history['evaluated_books'] = user_ratings['book_id'].tolist()
        
        # Évaluer
        rec_metrics = evaluate_recommendations(
            recommendations,
            user_history=user_history,
            all_books=books_df['id'].tolist() if books_df is not None else []
        )
        
        # Générer un rapport
        report = generate_evaluation_report(
            clustering_metrics=None,
            recommendation_metrics=rec_metrics
        )
        
        return jsonify({
            'status': 'success',
            'evaluation': {
                'recommendations': rec_metrics,
                'report': report
            },
            'user_id': user_id
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/debug/full/<int:user_id>', methods=['GET'])
def debug_full(user_id):
    """Debug complet avec évaluation"""
    try:
        # 1. Recharger les données
        load_data_from_db()
        
        # 2. Évaluer le clustering
        clustering_metrics = None
        if book_features_matrix is not None and book_clusters:
            clusters = []
            valid_indices = []
            for idx, book_id in index_to_book_id.items():
                cluster_id = book_clusters.get(book_id, -1)
                if cluster_id != -1:
                    clusters.append(cluster_id)
                    valid_indices.append(idx)
            
            if clusters:
                clusters_array = np.array(clusters)
                features_subset = book_features_matrix[valid_indices]
                clustering_metrics = evaluate_clustering(features_subset, clusters_array)
        
        # 3. Générer des recommandations
        recommendations = get_intelligent_recommendations(user_id)
        
        # 4. Évaluer les recommandations
        user_history = {'evaluated_books': []}
        if ratings_df is not None:
            user_ratings = ratings_df[ratings_df['user_id'] == user_id]
            user_history['evaluated_books'] = user_ratings['book_id'].tolist()
        
        rec_metrics = evaluate_recommendations(
            recommendations,
            user_history=user_history,
            all_books=books_df['id'].tolist() if books_df is not None else []
        )
        
        # 5. Rapport complet
        report = generate_evaluation_report(
            clustering_metrics=clustering_metrics,
            recommendation_metrics=rec_metrics
        )
        
        return jsonify({
            'status': 'success',
            'user_id': user_id,
            'clustering_evaluation': clustering_metrics,
            'recommendation_evaluation': rec_metrics,
            'full_report': report,
            'data_quality': data_quality_report,
            'n_recommendations': len(recommendations)
        })
        
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/user-profile/<int:user_id>', methods=['GET'])
def user_profile(user_id):
    """Récupère le profil utilisateur"""
    try:
        profile = get_user_profile(user_id)
        
        return jsonify({
            'status': 'success',
            'profile': {
                'user_id': user_id,
                'avg_rating': profile['avg_rating'],
                'total_ratings': profile['total_ratings'],
                'preferred_categories': list(profile['category_prefs'].keys())[:3],
                'preferred_authors': list(profile['author_prefs'].keys())[:5],
                'price_range': profile['preferred_price_range'],
                'last_updated': profile['last_updated']
            }
        })
    except Exception as e:
        print(f" Erreur route /user-profile: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/refresh-profile/<int:user_id>', methods=['POST'])
def refresh_profile(user_id):
    """Force le rafraîchissement du profil"""
    try:
        recalculate_user_profile(user_id)
        
        if user_id in user_features:
            return jsonify({
                'status': 'success',
                'message': f'Profil {user_id} rafraîchi',
                'profile': get_user_profile(user_id)
            })
        else:
            return jsonify({'status': 'error', 'message': 'Erreur rafraîchissement'}), 500
            
    except Exception as e:
        print(f" Erreur route /refresh-profile: {e}")
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/clusters/visualization', methods=['GET'])
def get_clusters_visualization():
    """Retourne la visualisation des clusters"""
    try:
        if book_features_matrix is None or not book_clusters:
            return jsonify({'status': 'error', 'message': 'Clustering non disponible'}), 400
        
        # Préparer les données pour la visualisation
        clusters = []
        valid_indices = []
        for idx, book_id in index_to_book_id.items():
            cluster_id = book_clusters.get(book_id, -1)
            if cluster_id != -1:
                clusters.append(cluster_id)
                valid_indices.append(idx)
        
        if not clusters:
            return jsonify({'status': 'error', 'message': 'Aucun cluster valide'}), 400
        
        clusters_array = np.array(clusters)
        features_subset = book_features_matrix[valid_indices]
        
        # Créer la visualisation
        save_path = "static/clusters_visualization.png"
        success = visualize_clusters(features_subset, clusters_array, save_path)
        
        if success:
            return jsonify({
                'status': 'success',
                'visualization_url': f'/{save_path}',
                'message': 'Visualisation créée avec succès'
            })
        else:
            return jsonify({'status': 'error', 'message': 'Erreur création visualisation'}), 500
            
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500

# %%
@app.route('/recommendations/fallback', methods=['GET'])
def get_fallback_recommendations():
    """Retourne des recommandations fallback"""
    try:
        recommendations = get_enhanced_fallback_recommendations()
        
        return jsonify({
            'status': 'success',
            'recommendations': recommendations,
            'count': len(recommendations),
            'algorithm': 'fallback'
        })
    except Exception as e:
        return jsonify({'status': 'error', 'message': str(e)}), 500   
app.run(host='0.0.0.0', port=5002, debug=True, use_reloader=False)










 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5002
 * Running on http://100.90.113.220:5002
Press CTRL+C to quit
